In [1]:
#Import necessary packages
import pandas as pd
import numpy as np
import math as math
import neighborhood as nbr
import seaborn as sns
import matplotlib.pyplot as plt
import time as time

import torch as torch
import warnings
warnings.filterwarnings('ignore')

Below is an import function written with some assistance from Josh Gorin to allow for an additional input into the neighborhood optimizer function

In [2]:
class Dataset(pd.DataFrame):
    def __init__(self, data:pd.DataFrame):
        super().__init__(data=data, index=data.index, columns=data.columns)
        assert self["TC"] is not None, "given dataset does not contain TC parameter."
        assert self["thr"] is not None, "given dataset does not contain thr parameter."
        assert self["ln(D/a^2)"] is not None, "given dataset does not contain ln(D/a^2) parameter."
        assert self["Fi"] is not None, "given dataset does not contain Fi parameter."

        self.np_TC = torch.tensor(self["TC"].values) 
        self.np_thr = torch.tensor(self["thr"].values) 
        self.np_lnDaa = torch.tensor(self["ln(D/a^2)"].values) 
        self.np_Fi_exp = torch.tensor(self["Fi"].values) 


In [3]:
class Objective:
    def __init__(self, fn, data:Dataset):
        self.fn = fn
        self.dataset = data

    def evaluate(self, X):
        return self.fn(X, self.dataset)

In [4]:
# Code written by Josh Gorin to allow for an additional input into the neighborhood optimizer function
import neighborhood as nbr
import pandas as pd


class Optimizer(nbr.Searcher):
    
    def __init__(self, dataset_path:str, objective_fn, limits:list[float], num_samp:int, 
                 num_resamp:int, names:list[str], maximize:bool=False, verbose:bool=True):
        
        if names == None:
            names = []
        
        
        super().__init__(
            objective_fn, 
            limits, 
            num_samp, 
            num_resamp, 
            names=names, 
            maximize=maximize, 
            verbose=verbose,
            
            
        )
        self.numEvals = 0
        self._objective = Objective(
            objective_fn, 
            Dataset(pd.read_csv(dataset_path))
        )

        
        self._numEvals = 0
        
    def update(self, num_iter=10):
        """
        tweaked from original codebase to pass in training data into objective function
        """

        for ii in range(num_iter):
            
            # If the first iteration, or every 10th... take a random sample
             
            if not self._sample or self._numEvals % 15 == 0:
                self._random_sample()
                self._numEvals = self._numEvals + 1 


            else:
                self._neighborhood_sample()

                self._numEvals = self._numEvals+ 1 
         
            # execute forward model for all samples in queue
            while self._queue:
                param = self._queue.pop()
                result = self._objective.evaluate(param)
                self._sample.append({
                    'param': param,
                    'result': result,
                    'iter': self._iter
                    })
             
            # prepare for next iteration
            self._sample.sort(key=lambda x: x['result'], reverse=self._maximize)
            self._iter += 1
            if self._verbose:
                print(self)

The below will be a cell for calculating D0 and D/a^2 from experimental data. This code is adapted from Marissa Tremblay's 2014 implementation of the Fechtig and Kalbitzer equations

In [5]:
def D0calc_MonteCarloErrors(expdata):
# Function for calculating D0 and D/a^2 from experimental data. Input should be a
# Pandas DataFrame with columns "TC", "thr",
# M, and, and delM, which correspond to heating temperature (deg C), 
# heating step duration (time in hours),
# M (measured concentration in cps, atoms, or moles), delM (same units)
    
    # Calculate diffusivities from the previous experiment
    TC = expdata.loc[:,"TC"].array
    thr = expdata.loc[:,"thr"].array
    M = expdata.loc[:,"M"].array
    delM = expdata.loc[:,"delM"].array

    #Check if units are in minutes and convert from hours to minutes if necesssary
    if thr[1]>4:
        thr = thr/60

    #Convert units
    TK = 273.15+TC
    tsec = thr*60*60
    Tplot = 1*10**4/TK
    nstep = len(M)
    cumtsec = np.cumsum(tsec)
    Si = np.cumsum(M)
    S = np.amax(Si)
    Fi = Si/S


    # initialize diffusivity vectors fore each Fechtig and Kalbitzer equation
    DR2_a = np.zeros([nstep])
    DR2_b = np.zeros([nstep])
    DR2_c = np.zeros([nstep])

    # Create the a list of times for each heating step
    diffti = cumtsec[1:]-cumtsec[0:-1]

    # Create a list of the gas fraction differences between steps
    diffFi = Fi[1:]-Fi[0:-1]


    # use equations 5a through c from Fechtig and Kalbitzer for spherical geometry
    # Fechtig and Kalbitzer Equation 5a, for cumulative gas fractions up to 10%
    # special case when i = 1; need to insert 0 for previous amount released

    DR2_a[0] = ( (Fi[0]**2 - 0.**2 )*math.pi/(36*(cumtsec[0])))


    # Equation 5a for all other steps

    DR2_a[1:] = ((Fi[1:])**2 - (Fi[0:-1])**2 )*math.pi/(36*(diffti))

    # Fechtig and Kalbitzer Equation 5b, for cumulative gas fractions between 10 and 90%

    DR2_b[0] = (1/((math.pi**2)*tsec[0]))*((2*math.pi)-((math.pi*math.pi/3)*Fi[0])\
                                           - (2*math.pi)*(np.sqrt(1-(math.pi/3)*Fi[0])))
    DR2_b[1:] = (1/((math.pi**2)*diffti))*(-(math.pi*math.pi/3)*diffFi \
                                           - (2*math.pi)*( np.sqrt(1-(math.pi/3)*Fi[1:]) \
                                            - np.sqrt(1 - (math.pi/3)*Fi[0:-1]) ))

    # Fechtig and Kalbitzer Equation 5c, for cumulative gas fractions greater than 90%
    DR2_c[1:] = (1/(math.pi*math.pi*diffti))*(np.log((1-Fi[0:-1])/(1-Fi[1:])))

    # Decide which equation to use based on the cumulative gas fractions from each step
    use_a = (Fi<= 0.1) & (Fi> 0.00000001)
    use_b = (Fi > 0.1) & (Fi<= 0.85)
    use_c = (Fi > 0.85) & (Fi<= 1.0)

    # Compute the final values
    DR2 = use_a*DR2_a + np.nan_to_num(use_b*DR2_b) + use_c*DR2_c

    # Compute uncertainties in diffusivity using a Monte Carlo simulation
    # Generates simulated step degassing datasets, such that each step of the 
    # experiment has a Gaussian distribution centered at M and with 1s.d. of
    # delM across the simulated datasets.Then recomputes diffusivities for each 
    # simulated dataset and uses the range of diffusivities for each step across
    # all simulated datasets to estimate uncertainty. 
    # make vector with correct diffusivites for each step

    n_sim = 30000 #number of simulations in the monte carlo
    MCsim = np.zeros([nstep,n_sim])#initialize matrix for simulated measurements


    
    
    for i in range(nstep):
        #Generate the simulated measurements
        MCsim[i,:] = np.random.randn(1,n_sim)*delM[i] + M[i]

    #compute cumulative gas release fraction for each simulation
    MCSi = np.cumsum(MCsim,0)
    MCS = np.amax(MCSi,0)
    MCFi = np.zeros([nstep,n_sim])
    delMCFi = np.zeros([nstep,1])
    MCFimean = np.zeros([nstep,1])



    for i in range(n_sim):
        MCFi[:,i] = MCSi[:,i]/np.amax(MCSi[:,i])
    for i in range(nstep):
        delMCFi[i] = (np.amax(MCFi[i,:],0) - np.amin(MCFi[i,:],0))/2
        MCFimean[i] = np.mean(MCFi[i,:],0)
      
    #Initialize vectors
    MCDR2_a = np.zeros([nstep,n_sim])
    MCDR2_b = np.zeros([nstep,n_sim])
    MCDR2_c = np.zeros([nstep,n_sim])
    MCdiffFi = np.zeros([nstep,n_sim])



    for m in range(1,nstep): #For step of each experiment...
        for n in range(n_sim):
            MCdiffFi[m,n] = MCFi[m,n] - MCFi[m-1,n] #calculate the fraction released at each step
    for n in range(n_sim): #For each first step of an experiment, insert 0 for previous amount released
        MCDR2_a[0,n] = ((MCFi[m,n])**2 - (MCFi[m-1,n])**2 )*math.pi/(36*(diffti[m-1]))
    for m in range(1,nstep): #Calculate fechtig and kalbitzer equations for each fraction
        for n in range(n_sim):
            MCDR2_a[m,n] = ( (MCFi[m,n])**2 - (MCFi[m-1,n])**2 )*math.pi/(36*(diffti[m-1]));
            MCDR2_b[m,n] = (1/((math.pi**2)*diffti[m-1]))*( -(math.pi*math.pi/3)* MCdiffFi[m,n] \
                            - (2*math.pi)*( np.sqrt(1-(math.pi/3)*MCFi[m,n]) \
                            -np.sqrt(1 - (math.pi/3)*MCFi[m-1,n]) ))
            MCDR2_c[m,n] = (1/(math.pi*math.pi*diffti[m-1]))*(np.log((1-MCFi[m-1,n])/(1-MCFi[m,n])));

    use_a_MC = (MCFi<= 0.1) & (MCFi> 0.00000001)
    use_b_MC = (MCFi > 0.1) & (MCFi<= 0.85)
    use_c_MC = (MCFi > 0.85) & (MCFi<= 1.0) 


    MCDR2 = use_a_MC*MCDR2_a + np.nan_to_num(use_b_MC*MCDR2_b) + use_c_MC*MCDR2_c

    MCDR2_uncert = np.zeros([nstep,1])
    for i in range(nstep):
        MCDR2_uncert[i,0] = np.std(MCDR2[i,:]) 


    return pd.DataFrame({"Tplot": Tplot,"Fi": MCFimean.ravel(),"Fi uncertainty": \
                               delMCFi.ravel(), "Daa": DR2,"Daa uncertainty": MCDR2_uncert.ravel(), \
                               "ln(D/a^2)": np.log(DR2),"ln(D/a^2)-del": np.log(DR2-MCDR2_uncert.ravel()), \
                               "ln(D/a^2)+del": np.log(DR2+MCDR2_uncert.ravel()) })

The following code is for use with the neighborhood algorithm as an objective function. It takes in diffusion kinetics (X) and experimental data (data) and returns a misfit between the two.

In [6]:
def forwardModelKinetics(kinetics,expData): 
    # kinetics: (Ea, lnd0aa_x, fracs_x). To make this compatible with other functions, if there are x fracs, input x-1 fractions, and the code will determine the
    # final fraction.
    
    R = 0.008314 #gas constant
    torch.pi = torch.acos(torch.zeros(1)).item() * 2
     # Parameters that need to be read in (These I'll likely read from a file eventually
    # But we'll read the data from above just to test the function for now...)
    if len(kinetics) <= 3:
        ndom = 1
    else:
        ndom = (len(kinetics))//2

    # Make a subset of X, removing the Ea so that we have an even number of elements
    temp = kinetics[1:]
 
    if type(expData) is tuple:
        TC = expData[0]
        thr = expData[1]
        lnDaa = expData[2]
        Fi = expData[3]

    else:
        TC = expData.np_TC
        thr = expData.np_thr
        lnDaa = expData.np_lnDaa
        Fi = expData.np_Fi_exp
    
    # Grab the parameters from the input
    lnD0aa = torch.tile(temp[0:ndom],(len(thr),1)) #lnD0aa = np.tile(lnD0aa,(len(thr),1))
    fracstemp = temp[ndom:]

    fracs = torch.tile(torch.concat((fracstemp,1-torch.sum(fracstemp,axis=0,keepdim=True)),axis=-1),(len(thr),1))
    Ea = torch.tile(kinetics[0],(len(thr),ndom)) # This is an Ea for each domain

    # Put variables in correct units

    tsec = torch.tile(torch.reshape(thr*3600,(-1,1)),(1,Ea.shape[1])) #This is a complicated way of getting tsec into a numdom x numstep matrix for multiplication
    cumtsec = torch.tile(torch.reshape(torch.cumsum(thr*3600,dim=0),(-1,1)),(1,Ea.shape[1])) #Same as above, but for cumtsec                                                         
    TK = torch.tile(torch.reshape((TC + 273.15),(-1,1)),(1,Ea.shape[1])) #This is a complicated way of turning TC from a 1-d array to a 2d array and making two column copies of it
    

    Daa = torch.exp(lnD0aa)*torch.exp(-Ea/(R*TK))
    
    # Pre-allocate fraction and Dtaa
    f = torch.zeros(Daa.shape)
    Dtaa = torch.zeros(Daa.shape)
    DaaForSum = torch.zeros(Daa.shape)
    
    
    #Calculate D
    DaaForSum[0,:] = Daa[0,:]*tsec[0,:]
    DaaForSum[1:,:] = Daa[1:,:]*(cumtsec[1:,:]-cumtsec[0:-1,:])

    Dtaa = torch.cumsum(DaaForSum, axis = 0)

    f = (6/(math.pi**(3/2)))*torch.sqrt((math.pi**2)*Dtaa)
    f[f>=0.1] = (6/(torch.pi**(3/2)))*torch.sqrt((torch.pi**2)*Dtaa[f>=0.1])-(3/(torch.pi**2))* \
            ((torch.pi**2)*Dtaa[f>=0.1])
    f[f>=0.9] = 1 - (6/(torch.pi**2))*torch.exp(-(torch.pi**2)*Dtaa[f>=0.9])

    # If any member of f is preceeded by a value greater than it, set equal to 1 (we've reached total gas released)
    f[1:,:][f[1:,:]<f[0:-1,:]] = 1
    f[1:,:][f[0:-1,:]==1] = 1

    # Multiply each gas realease by the percent gas located in each domain

    f_MDD = f*fracs

    # Calculate the total gas released at each step from each gas fraction
    sumf_MDD = torch.sum(f_MDD,axis=1)
    

    #Calculate the apparent Daa from the MDD using equations of Fechtig and Kalbitzer
    Daa_MDD_a = torch.zeros(sumf_MDD.shape)
    Daa_MDD_b = torch.zeros(sumf_MDD.shape)
    Daa_MDD_c = torch.zeros(sumf_MDD.shape)
   
    # use equations 5a through c from Fechtig and Kalbitzer for spherical geometry
    # Fechtig and Kalbitzer Equation 5a, for cumulative gas fractions up to 10%
    # special case when i = 1; need to insert 0 for previous amount released

    # Rewrite the cumtsec as just one column to make for easier calculations below 
    # and because we don't need to use on three domains separately anymore
    cumtsec = cumtsec[:,0]
    diffti = cumtsec[1:]-cumtsec[0:-1]
    diffFi = sumf_MDD[1:]-sumf_MDD[0:-1]

    Daa_MDD_a[0] = ( (sumf_MDD[0]**2 - 0.**2 )*torch.pi/(36*(cumtsec[0])))


    # Equation 5a for all other steps

    Daa_MDD_a[1:] = ((sumf_MDD[1:])**2 - (sumf_MDD[0:-1])**2 )*math.pi/(36*(diffti))

    # Fechtig and Kalbitzer Equation 5b, for cumulative gas fractions between 10 and 90%
    Daa_MDD_b[0] = (1/((torch.pi**2)*tsec[0,0]))*((2*torch.pi)-((math.pi*math.pi/3)*sumf_MDD[0])\
                                           - (2*math.pi)*(torch.sqrt(1-(math.pi/3)*sumf_MDD[0])))
    Daa_MDD_b[1:] = (1/((math.pi**2)*diffti))*(-(math.pi*math.pi/3)*diffFi \
                                           - (2*math.pi)*( torch.sqrt(1-(math.pi/3)*sumf_MDD[1:]) \
                                            - torch.sqrt(1 - (math.pi/3)*sumf_MDD[0:-1]) ))

    # Fechtig and Kalbitzer Equation 5c, for cumulative gas fractions greater than 90%
    Daa_MDD_c[1:] = (1/(math.pi*math.pi*diffti))*(torch.log((1-sumf_MDD[0:-1])/(1-sumf_MDD[1:])))

    # Decide which equation to use based on the cumulative gas fractions from each step
    use_a = (sumf_MDD<= 0.1) & (sumf_MDD> 0.00000001)
    use_b = (sumf_MDD > 0.1) & (sumf_MDD<= 0.85)
    use_c = (sumf_MDD > 0.85) & (sumf_MDD<= 1.0)
    Daa_MDD = use_a*Daa_MDD_a + torch.nan_to_num(use_b*Daa_MDD_b) + use_c*Daa_MDD_c
    

    lnDaa_MDD = torch.log(Daa_MDD)
    
    return (TC,lnDaa,sumf_MDD)
    #return pd.DataFrame({"TC": TC.ravel(),"ln(D/a^2)_MDD": lnDaa_MDD.ravel(),"Fi_MDD": sumf_MDD.ravel()}) # Return a dataframe with the results forward modeled   

In [7]:
def objectiveFunction(X,data):
    torch.pi = torch.acos(torch.zeros(1)).item() * 2
    # Below here will eventually get turned into a function
    # Code written by Marissa Tremblay and modified/transcribed into Python by Drew Gorin.
    #Last modified 1.2023.

    # This function calculates the fraction of gas released from each domain
    # in an MDD model during the heating schedule used in the diffusion
    # experiment. Then the fractions released from each domain are combined in
    # proportion to one another as specified by the MDD model, and the
    # diffusivity of each step is calculated. A residual is calculated as the
    # sum of absolute differences between the observed and modeled release
    # fractions over all steps.
    
    #Time both of these
    #X = torch.from_numpy(X)
    X = torch.as_tensor(X)
    # Unpack the parameters and spit out a high misfit value if constraints are violated
    if len(X) <= 3:
        ndom = 1
    else:
        ndom = (len(X))//2


    # Grab the other parameters from the input
    
    temp = X[1:]
    lnD0aa = temp[0:ndom]
    fracstemp = temp[ndom:] 
    sumTemp = (1-torch.sum(fracstemp,axis=0,keepdim = True))
    fracs = torch.concat((fracstemp,sumTemp),dim=-1)
    # Report high misfit values if conditions are not met

    
    for i in range(len(lnD0aa)-1):
        if lnD0aa[i+1]>lnD0aa[i]:
            return 10**10
        
    fwdModelResults = forwardModelKinetics(X,data)

    # Parameters that need to be read in (These I'll likely read from a file eventually
    # But we'll read the data from above just to test the function for now...)

    TC = data.np_TC #data["TC"].to_numpy()
    thr = data.np_thr#["thr"].to_numpy()
    lnDaa = data.np_lnDaa#["ln(D/a^2)"].to_numpy()
    Fi_exp = data.np_Fi_exp#["Fi"].to_numpy()
    Fi_MDD = fwdModelResults[2]#fwdModelResults["Fi_MDD"].to_numpy()

    
    #Calculate the Fraction released for each heating step
    TrueFracFi = (Fi_exp[1:]-Fi_exp[0:-1])
    TrueFracFi = torch.concat((torch.unsqueeze(Fi_exp[0],dim=-1),TrueFracFi),dim=-1)
    

    trueFracMDD = Fi_MDD[1:]-Fi_MDD[0:-1]
    trueFracMDD = torch.concat((torch.unsqueeze(Fi_MDD[0],dim=-1),trueFracMDD),dim=-1)
    # Calculate the L1 loss 
    misfit = torch.absolute(TrueFracFi-trueFracMDD)

    # Assign penalty for each step if the model runs out of gas before the experiment did

    # Add a misfit penalty of 1 for each heating step that ran out of gas early
    misfit[trueFracMDD==0] = 1
    
    if torch.round(Fi_MDD[-1],decimals=4) != 1:
         return 10**10
#     if Fi_MDD[-2] >= 1:
#          return 10**10

    if Fi_MDD[-2] >= 1:
         return 10**2
   # Return the sum of the residuals
    return torch.sum(misfit)

In [8]:
def plotModelResults(fwdModel,expData):
    # Calculate the temp in 10000/TK (Standard plotting units for this field)
    expData["10000/TK"] = 10000/(expData["TC"]+273.15)
    fwdModel["10000/TK"] = 10000/(fwdModel["TC"]+273.15)
    fwdModel["MDDFi_Cum"] = np.cumsum(fwdModel["Fi_MDD"])
    expData["Fi_Cum"] = np.cumsum(expData["Fi"])
    
    
    plt.figure();
    plt.subplot(1,2,1)
    sns.scatterplot(data = expData, x = "10000/TK", y = "ln(D/a^2)")
    sns.scatterplot(data = fwdModel, x = "10000/TK",y="ln(D/a^2)_MDD")
    plt.legend(["Experiment","Model"])
    plt.subplot(1,2,2)
    sns.scatterplot(data = expData, x =expData.index, y = "Fi")
    sns.scatterplot(data = fwdModel, x =fwdModel.index, y="Fi_MDD")
    plt.legend(["Experiment","Model"])


In [9]:
# This is the MAIN function

#Code written by Marissa Tremblay and transcribed into Python/modified by Drew Gorin. Last modified 1.2023

#This m-file is used to fit an MDD model to stepwise degassing diffusion
#experiment data. It is currently set up for only one isotope. The number
#of domains is allowed to vary. The activation energy is assumed to be the
#same across domains while the pre-exponential factor (D0/a^2) and the
#fraction of gas in each domain varies. Needs the companion functions
#D0calc_MonteCarloErros.m and TremblayMDD.m.

# USER INPUT HERE (should be a csv with no header)
nameOfInputCSVFile = "3Domains.csv"
nameOfExperimentalResultsFile = "data4Optimizer.csv"

## USER SHOULD NOT MODIFY LINES BELOW THIS


expData = pd.read_csv(nameOfInputCSVFile,header=None)
    
#If extra columns get read in, trim them down to just 3
if expData.shape[1] >=4:
    expData = expData.loc[:,1:4]
    
# Name the columsn of the iput data
expData.columns = ["TC", "thr","M", "delM"]


# Calculate Daa from experimental results
expResults = D0calc_MonteCarloErrors(expData)

# Combine the diffusion parameters with the experimental setup (T, thr, M, delM)
# to get a final dataframe that will be passed into the optimizer
diffusionExperimentResults = expData.join(expResults)

# Write dataframe to a .csv file
diffusionExperimentResults.to_csv(nameOfExperimentalResultsFile)

In [10]:
%%time
# Now I'll try the same optimization with the built in scipy global optimizer to see if it does a better job or is faster.
from scipy import optimize
bounds = [(70,110),(0,25),(0,25),(0,25),(10**(-10),1),(10**(-10),1)]
ranges = bounds
args = (torch.tensor(diffusionExperimentResults["TC"]),torch.tensor(diffusionExperimentResults["thr"]), \
        torch.tensor(diffusionExperimentResults["ln(D/a^2)"]),torch.tensor(diffusionExperimentResults["Fi"]))

#results = optimize.brute(objectiveFunction4PythonOptimizer,bounds,args = args,locally_biased=True,eps = 1e-7)

results = optimize.brute(objectiveFunction4PythonOptimizer,ranges,args = args,Ns=13,full_output = False)
#data = Dataset(diffusionExperimentResults)
#fwdModel = forwardModelKinetics(torch.tensor(results.x),data)
# plotModelResults(fwdModel,data)

results
# diffusionExperimentResults
# results

NameError: name 'objectiveFunction4PythonOptimizer' is not defined

In [13]:
# I wrote this to accomodate a python optimizer at one point, but this is now out of date. don't use
data = pd.read_csv("out",delimiter=',')
data = Dataset(data)


def objectiveFunction4PythonOptimizer(X):

    torch.pi = torch.acos(torch.zeros(1)).item() * 2
    # Below here will eventually get turned into a function
    # Code written by Marissa Tremblay and modified/transcribed into Python by Drew Gorin.
    #Last modified 1.2023.

    # This function calculates the fraction of gas released from each domain
    # in an MDD model during the heating schedule used in the diffusion
    # experiment. Then the fractions released from each domain are combined in
    # proportion to one another as specified by the MDD model, and the
    # diffusivity of each step is calculated. A residual is calculated as the
    # sum of absolute differences between the observed and modeled release
    # fractions over all steps.
    
    #Time both of these
    #X = torch.from_numpy(X)

    X = torch.as_tensor(X)
    # Unpack the parameters and spit out a high misfit value if constraints are violated
    if len(X) <= 3:
        ndom = 1
    else:
        ndom = (len(X))//2


    # Grab the other parameters from the input
    
    temp = X[1:]
    lnD0aa = temp[0:ndom]
    fracstemp = temp[ndom:] 
    sumTemp = (1-torch.sum(fracstemp,axis=0,keepdim = True))
    fracs = torch.concat((fracstemp,sumTemp),dim=-1)

    # Report high misfit values if conditions are not met

    
    for i in range(len(lnD0aa)-1):
        if lnD0aa[i+1]>lnD0aa[i]:
            return 10**10
        
    fwdModelResults = forwardModelKinetics(X,data)

    # Parameters that need to be read in (These I'll likely read from a file eventually
    # But we'll read the data from above just to test the function for now...)

    TC = data.np_TC
    thr = data.np_thr
    lnDaa = data.np_lnDaa
    Fi_exp = data.np_Fi_exp
    Fi_MDD = fwdModelResults[2]

    
    #Calculate the Fraction released for each heating step
    TrueFracFi = (Fi_exp[1:]-Fi_exp[0:-1])
    TrueFracFi = torch.concat((torch.unsqueeze(Fi_exp[0],dim=-1),TrueFracFi),dim=-1)
    

    trueFracMDD = Fi_MDD[1:]-Fi_MDD[0:-1]
    trueFracMDD = torch.concat((torch.unsqueeze(Fi_MDD[0],dim=-1),trueFracMDD),dim=-1)
    # Calculate the L1 loss 
    misfit = torch.absolute(TrueFracFi-trueFracMDD)

    # Assign penalty for each step if the model runs out of gas before the experiment did

    # Add a misfit penalty of 1 for each heating step that ran out of gas early
    misfit[trueFracMDD==0] = 1
    
    if torch.round(Fi_MDD[-1],decimals=4) != 1:
         return 10**10
#     if Fi_MDD[-2] >= 1:
#          return 10**10

    if Fi_MDD[-2] >= 1:
         return 10**2
   # Return the sum of the residuals

    return torch.sum(misfit).item()

# Some Notes for Josh 

The inputs to the optimizer have some constraints, and you can see that I've badly enforced them by assigning high misfit values if they aren't satisfied. 

## Model has $2* num Domains +1$ parameters. They are as follows:

#### Ea: Activation energy in kj/mol

#### $\ln(D_0/a^2)$: in (1/s). There is one of these per domain

#### Fraction of total gas (There are $numDom-1$ of these because $\sum_{i=1}^{\#Dom} Frac_i = 1$, so if you know $numDom-1$ of them, you know the value of last Frac).

## The input parameters vector is as follows: [(Ea, lnd0aa1, lnd0aa2,...lnd0aa_x,Frac_1,Frac_2,...Frac_(numDom-1))]


### The constraints on the inputs are as follows:

#### $\ln(D_0/a^2)_1 > ln(D_0/a^2)_2 > ... ln(D_0/a^2)_{numDom}  $

#### $Fracs_1 + Fracs_2 + Fracs3... + Fracs_ndom = 1  $ However, we only stick the first numDom-1 into the optimizer, since the last Frac is determined by the other numDom-1 params

#### Fi_MDD[-1] must = 1


Note: The data we're playing with is a synthetic dataset I created with known values for all of these parameters and it has 3 domains. When solving for real data, we won't know the number of domains, but this is a good way for us to make sure the optimizer is behaving properly.

In [12]:
# I wrote this gross while loop to try to test out the possible values of the model


threshold = 0.01
num_dim = 6
num_sampToTry = range(7,21)
num_ResampToTry = range(3,20)
misfitVals = np.ones([101,101])*10*11
numIterations4Save = np.ones([100,101])*10*11
durations = np.ones([101,101])*10*11



for i in num_sampToTry:
    j=i
    while j <=30:

        misfit = 10**11
        counter = 0
        print("num_samp = " +str(j))
        print("num_resamp = " +str(i))
        srch = Optimizer(
                objective_fn=objectiveFunction,
                names = ["Ea","LnD0aa1","LnD0aa2","LnD0aa3","Frac1","Frac2"],
                limits=[(70,110),(0,25),(0,25),(0,25),(10**(-10),1),(10**(-10),1)], 
                dataset_path = nameOfExperimentalResultsFile,
                num_samp=j, #number of random samples taken at each iteration
                num_resamp=i, #number of best Voronoi polygons sampled at each iteration-- must be smaller than num_samp
                maximize=False,
                verbose=False
                )
        start_time = time.time()
        didNotConverge = 0
        numIters = 0
        while (misfit >= threshold) and (didNotConverge == 0): # While misfit is below the threshold and we haven't failed to converge
            srch.update(10)
            misfit = srch.sample[0]["result"]
            numIters = numIters+10
            if (numIters > 100) and (srch.sample[0]["result"] == srch.sample[99]["result"]):
                didNotConverge = 1
                print("Did not converge")

            elif numIters > 3000:
                didNotConverge = 1
                print("Did not converge")

       
        misfitVals[i,j] = misfit
        durations[i,j] = (time.time() - start_time)
        numIterations4Save[i,j] = numIters
        j=j+1
        if didNotConverge == 0:
            print("Converged!")

        
        
#     # Extract the best fitting model with this while loop
#         exitFlag = 0
#         counter = 0
#         while exitFlag == 0:
#             if srch.sample[counter]['iter'] == n_iter-1:
#                 bestFit = srch.sample[counter]
#                 exitFlag = 1
#             else:
#                 counter += 1

#         misfitVals[i-1,j-1] = srch.sample[counter]["result"]

# fwdModel = forwardModelKinetics(srch.sample[counter]['param'],diffusionExperimentResults)
# plotModelResults(fwdModel,diffusionExperimentResults)
# bestFit


np.savetxt("misfitVals.csv", misfitVals,delimiter = ',')
np.savetxt("numiters.csv", numIterations4Save, delimiter = ',')
np.savetxt("durations.csv", durations, delimiter = ',')


num_samp = 7
num_resamp = 7


KeyboardInterrupt: 

In [ ]:
np.savetxt("misfitVals7and8.csv", misfitVals,delimiter = ',')
np.savetxt("numiters7and8.csv", numIterations4Save, delimiter = ',')
np.savetxt("durations7and8.csv", durations, delimiter = ',')

In [ ]:
from skopt.plots import plot_gaussian_process

In [ ]:
from skopt import gp_minimize

res = gp_minimize(objectiveFunction4PythonOptimizer,  # the function to minimize
                  bounds,      # the bounds on each dimension of x
                  acq_func="gp_hedge",      # the acquisition function
                  n_calls=100,         # the number of evaluations of f
                  n_random_starts=50,  # the number of random initialization points
                  noise=0.1**2,       # the noise level (optional)
                  random_state=1234,
                  #InitialPointGenerator = "sobol",
                  verbose = True)   # the random seed

res




In [14]:
data = pd.read_csv("out",delimiter=',')
data = Dataset(data)
def objectiveWrapper(X):

    return np.array([objectiveFunction4PythonOptimizer(i) for i in X])
    

    

In [29]:
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx
from pyswarms.utils.plotters import (plot_cost_history, plot_contour, plot_surface)


scoopydo = ps.single.LocalBestPSO(n_particles=30, 
                                        dimensions = 6, 
                                        options = {'c1': 3.9363349786571726, 'c2': 9.611193885252797, 'w': 8.976464260292225, 'k': 1, 'p': 1},
                                        bounds= (np.array([70,0,0,0,10**(-10),10**(-10)]),np.array([110,25,25,25,1,1])),
                                        bh_strategy='periodic',
                                        velocity_clamp=None,
                                        vh_strategy='unmodified',
                                        ftol= float("-inf"), 
                                        init_pos=None,
                                        
                                        )

optimizer = scoopydo.optimize(objectiveWrapper,iters = 5000)

2023-02-02 10:42:39,688 - pyswarms.single.local_best - INFO - Optimize for 5000 iters with {'c1': 3.9363349786571726, 'c2': 9.611193885252797, 'w': 8.976464260292225, 'k': 1, 'p': 1}
pyswarms.single.local_best: 100%|████████████████████|5000/5000, best_cost=0.909
2023-02-02 10:46:32,477 - pyswarms.single.local_best - INFO - Optimization finished | best cost: 0.9089250609983927, best pos: [1.02000000e+02 1.50000000e+01 1.10000000e+01 7.00000000e+00
 6.88417898e-01 7.36371631e-02]


In [22]:
%%time
from pyswarms.utils.search import RandomSearch
from pyswarms.utils.search import GridSearch

options = {'c1': [.1, 20],
'c2': [.1, 20],
'w' : [.1, 10],
'k' : [.1, 20],
'p' : 1}

g = RandomSearch(ps.single.GlobalBestPSO, n_particles=1000, dimensions=6,
options=options, objective_func=objectiveWrapper, n_selection_iters=2400,\
                 iters = 50, bounds= \
                 (np.array([70,0,0,0,10**(-10),10**(-10)]),\
                  np.array([110,25,25,25,1,1])))

best_score, best_options = g.search()



2023-02-01 11:59:08,170 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 7.372257265427903, 'c2': 12.36014042090018, 'w': 7.7018506894801355, 'k': 9, 'p': 1}
pyswarms.single.global_best: 100%|████████████████████████|50/50, best_cost=0.34
2023-02-01 11:59:46,144 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.33984262356251244, best pos: [109.12551228  20.1112569   17.05464298   8.63970048   0.50848715
   0.43923874]
2023-02-01 11:59:46,153 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 16.742683100585253, 'c2': 7.935171660940483, 'w': 8.78556664891918, 'k': 7, 'p': 1}
pyswarms.single.global_best: 100%|███████████████████████|50/50, best_cost=0.415
2023-02-01 12:00:08,078 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.4146873495158584, best pos: [7.94943492e+01 1.10258414e+01 6.18185901e+00 2.83738949e+00
 8.05504520e-01 7.04761008e-02]
2023-02-01 12:00:08,087 - pyswarms.single.gl

pyswarms.single.global_best: 100%|███████████████████████|50/50, best_cost=0.488
2023-02-01 12:08:02,433 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.48774083366309795, best pos: [81.62604849 11.35653701  8.27083093  3.39428402  0.47665062  0.2206279 ]
2023-02-01 12:08:02,442 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 17.420036738877396, 'c2': 9.192320928043635, 'w': 8.048108489043488, 'k': 13, 'p': 1}
pyswarms.single.global_best: 100%|███████████████████████|50/50, best_cost=0.287
2023-02-01 12:08:28,738 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.2874966527388554, best pos: [99.35928582 17.38326328 15.10445279  6.67190726  0.2996248   0.67097756]
2023-02-01 12:08:28,747 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 12.17688744394375, 'c2': 17.67331125654729, 'w': 8.811016772344793, 'k': 7, 'p': 1}
pyswarms.single.global_best: 100%|███████████████████████|50/50, best

2023-02-01 12:15:57,765 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 7.740798009312018, 'c2': 1.8565907430893114, 'w': 2.6198137220512994, 'k': 1, 'p': 1}
pyswarms.single.global_best: 100%|███████████████████████|50/50, best_cost=0.324
2023-02-01 12:16:24,820 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.32400203270195793, best pos: [102.72173789  20.39069612  16.73208683   7.42154045   0.13688832
   0.8140975 ]
2023-02-01 12:16:24,828 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 18.100550450672753, 'c2': 7.207238871687294, 'w': 3.1487585299112744, 'k': 10, 'p': 1}
pyswarms.single.global_best: 100%|███████████████████████|50/50, best_cost=0.496
2023-02-01 12:16:50,217 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.4955310366447119, best pos: [9.73185582e+01 1.66078562e+01 1.07704665e+01 6.14816988e+00
 8.76144846e-01 5.77500014e-02]
2023-02-01 12:16:50,226 - pyswarms.singl

pyswarms.single.global_best: 100%|███████████████████████|50/50, best_cost=0.867
2023-02-01 12:24:57,710 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.8672454638566104, best pos: [82.31147288 11.6742108   4.84495404  3.25202954  0.35551182  0.64373126]
2023-02-01 12:24:57,719 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 3.733182481635627, 'c2': 10.415642306810687, 'w': 7.452258229317279, 'k': 3, 'p': 1}
pyswarms.single.global_best: 100%|█████████████████████████|50/50, best_cost=1.4
2023-02-01 12:25:15,426 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 1.404257083752341, best pos: [71.32314722  4.20994612  2.28806849  1.37620979  0.14360419  0.83908335]
2023-02-01 12:25:15,434 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 12.493720838182462, 'c2': 15.926383736966132, 'w': 7.7328990968902165, 'k': 13, 'p': 1}
pyswarms.single.global_best: 100%|███████████████████████|50/50, bes

2023-02-01 12:32:23,963 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 9.059080303163856, 'c2': 12.388225821766225, 'w': 3.042596881613206, 'k': 1, 'p': 1}
pyswarms.single.global_best: 100%|███████████████████████|50/50, best_cost=0.342
2023-02-01 12:32:52,993 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.34214467340600996, best pos: [92.19728118 13.88339669 13.07251653  5.46964406  0.61869345  0.31804339]
2023-02-01 12:32:53,002 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 18.58112421178481, 'c2': 12.855793459736233, 'w': 3.3505959431496275, 'k': 12, 'p': 1}
pyswarms.single.global_best: 100%|███████████████████████|50/50, best_cost=0.726
2023-02-01 12:33:21,858 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.7255501594611586, best pos: [103.71303534  18.37990296  10.74521741   7.61531338   0.42832673
   0.50742111]
2023-02-01 12:33:21,867 - pyswarms.single.global_best - INFO

...

2023-02-02 06:04:06,362 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 16.547569505753607, 'c2': 12.478492311191255, 'w': 7.791223297222771, 'k': 7, 'p': 1}
pyswarms.single.global_best: 100%|████████████████████████|50/50, best_cost=1.04
2023-02-02 06:04:26,393 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 1.0440701698158454, best pos: [73.32619322 10.67588294  1.65763046  1.65726797  0.66558968  0.38314096]
2023-02-02 06:04:26,402 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 19.647709576512977, 'c2': 6.681951897852544, 'w': 6.853186157547813, 'k': 12, 'p': 1}
pyswarms.single.global_best: 100%|███████████████████████|50/50, best_cost=0.383
2023-02-02 06:04:48,225 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.3832650419209689, best pos: [89.8389106  13.00557837 12.0106102   4.96551164  0.79062865  0.10234605]
2023-02-02 06:04:48,234 - pyswarms.single.global_best - INFO - Optim

pyswarms.single.global_best: 100%|███████████████████████|50/50, best_cost=0.823
2023-02-02 06:11:50,345 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.8232287987243228, best pos: [7.58544586e+01 9.17319853e+00 3.11390132e+00 2.28199368e+00
 4.45920635e-01 2.79552106e-02]
2023-02-02 06:11:50,354 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 5.282352223188072, 'c2': 11.334422379316235, 'w': 5.999584440720611, 'k': 10, 'p': 1}
pyswarms.single.global_best: 100%|███████████████████████|50/50, best_cost=0.279
2023-02-02 06:12:19,989 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.2788657004527662, best pos: [94.56709356 15.47320449 14.67263056  5.91763831  0.13557141  0.79847348]
2023-02-02 06:12:19,997 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 15.947201744805534, 'c2': 1.1787237804696624, 'w': 5.468591470800199, 'k': 19, 'p': 1}
pyswarms.single.global_best: 100%|██████████████

2023-02-02 06:20:11,117 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 15.148918793547297, 'c2': 17.529450170184074, 'w': 7.6697390244054, 'k': 18, 'p': 1}
pyswarms.single.global_best: 100%|███████████████████████|50/50, best_cost=0.327
2023-02-02 06:20:48,003 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.32683772948767087, best pos: [109.91900222  19.83589747  16.98045285   8.51910017   0.59733718
   0.29039554]
2023-02-02 06:20:48,011 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 2.4849358216471527, 'c2': 5.5210037394065985, 'w': 3.067905634674814, 'k': 18, 'p': 1}
pyswarms.single.global_best: 100%|███████████████████████|50/50, best_cost=0.362
2023-02-02 06:21:13,756 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.3620976867177341, best pos: [89.63708807 13.10479748 12.90499821  4.98484456  0.15133541  0.7423235 ]
2023-02-02 06:21:13,765 - pyswarms.single.global_best - INFO

pyswarms.single.global_best: 100%|███████████████████████|50/50, best_cost=0.281
2023-02-02 06:29:00,181 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.2813740742246973, best pos: [90.84208306 13.81039476 12.1351232   5.10277673  0.67045882  0.17898671]
2023-02-02 06:29:00,191 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 4.526296483406738, 'c2': 1.733951769051051, 'w': 4.823518073111376, 'k': 10, 'p': 1}
pyswarms.single.global_best: 100%|███████████████████████|50/50, best_cost=0.674
2023-02-02 06:29:29,905 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.6738004913090381, best pos: [102.97539811  22.31400355  16.72174479   7.3376679    0.33221178
   0.54673906]
2023-02-02 06:29:29,914 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 19.90582903805145, 'c2': 16.163086313334617, 'w': 2.4888280281087805, 'k': 8, 'p': 1}
pyswarms.single.global_best: 100%|███████████████████████|50/5

CPU times: user 18h 28min 38s, sys: 3min 1s, total: 18h 31min 40s
Wall time: 18h 30min 50s


In [24]:
best_options

{'c1': 3.9363349786571726,
 'c2': 9.611193885252797,
 'w': 8.976464260292225,
 'k': 2,
 'p': 1}